## 투빅스 18기 Week1 Framework 과제 - 18기 이다인

## 1. 
#### 패키지와 라이브러리 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

## 2.
#### 하드웨어 설정하기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
#위 코드가 필요한 이유는 학습을 위한 실험에서 무작위성을 컨트롤 하기 위함
print(device + " is available")

cpu is available


## 3.
#### 학습에 사용될 파라미터 설정하기

learning_rate = 학습률   
epoch = 전체 트레이닝 셋이 신경망을 통과한 횟수  
num_classes = 출력 데이터의 크기  
batch_size = 전체 트레이닝 셋을 여러 작은 그룹으로 나누었을 때 하나의 소그룹에 속하는 데이터의 수  

In [3]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

## 4.
#### MNIST 데이터셋을 로드하여 train,test 데이터셋을 배치형태로 변환시켜주기

In [4]:
#MNIST의 데이터셋을 다운로드 하였으며, ToTensor()를 이용하여 데이터를 0~255 값에서 0~1사이의 값으로 변환하였다.
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

#MNIST의 데이터셋을 다운로드 하였으며, ToTensor()를 이용하여 데이터를 0~255 값에서 0~1사이의 값으로 변환하였다.
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
#train_loader와 test_loader를 통해 MNIST에서 불러온 데이터를 batch형태로 변환시켜주기
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

## 5.
#### input size 파악하기

In [5]:
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

## 6. 
#### 모델 구축 및 손실 함수와 옵티마이저 설정

In [6]:
class ConvNet(nn.Module): #컨볼루션 신경망 정의
  def __init__(self): 
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input channel = 1, filter = 10, kernel size = 5, zero padding = 0, stribe = 1
        #input size = 28x28에서 컨볼루션 신경망 출력사이즈 공식을 사용한 후 mxpooling하여 12x12로 변환
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input channel = 10, filter = 20, kernel size = 5, zero padding = 0, stribe = 1
        #12x12에서 컨볼루션 신경망 출력사이즈 공식을 사용한 후 maxpooling하여 4x4로 변환
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.mp = nn.MaxPool2d(2) 
        # 오버피팅과 하드웨어 리소스 낭비를 방지하기 위해 맥스풀링
        self.fc1 = nn.Linear(320,100) 
        # 100개의 라벨을 출력하도록 하는 fully connected layer
        self.fc2 = nn.Linear(100,10) 
        # 100개를 다시 10개의 라벨으로 출력하도록 하는 fully connected layer

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # 위에서 정의된 convolution layer 1에 relu를 씌움
        x = F.relu(self.mp(self.conv2(x))) # 위에서 정의된 convolution layer 2에 relu를 씌움
        #relu 함수는 기울기 소실 문제 방지를 위하여 사용하는 활성화 함수
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) # 전결합층을 위하여 flatter시키는 작업
        x = self.fc1(x) #위에서 정의된 첫번째 fully connected layer에 삽입
        x = self.fc2(x) #위에서 정의된 두번째 fully connected layer에 삽입
        return F.log_softmax(x) #logsoftmax 적용 : 입력값을 0에서 1사이의 값으로 정규화시키는 softmax함수에 log를 취한 함수로 더 빠르고 안정적이다
    
 
model = ConvNet().to(device) #GPU에서 연산 수행
#손실함수(=Cost Function)와 optimizer 선택
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## 7.
#### epoch 수 만큼 모델 학습 반복

In [7]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        #모든 model의 gradient 값을 0으로 초기화, 만일 초기화 하지 않으면 미분값이 계속 누적되기 때문
        hypothesis = model(data) #forward연산 수행
        cost = criterion(hypothesis, target) # 비용함수로 output과 target의 loss 계산
        cost.backward() 
        optimizer.step() #모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader) # loss 값을 변수에 누적한 후 trainloader의 개수로 나눔
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12992\3650849640.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1] cost = 0.31730783
[Epoch:    2] cost = 0.114742726
[Epoch:    3] cost = 0.0868237242
[Epoch:    4] cost = 0.0750293583
[Epoch:    5] cost = 0.0662271008


## 8.
#### 모델 성능 테스트

In [8]:
model.eval() # evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 차단시키는 함수
with torch.no_grad(): #grad 해제
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] #가장 높은 값 출력
        total += len(target) 
        correct += (preds==target).sum().item() # 예측값과 실제값 비교
        
    print('Test Accuracy: ', 100.*correct/total, '%')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12992\3650849640.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.57 %
